To have the best recommandations for our costumer, Monica Villarreal, we will apply a User based collaborative filtering.

We will, for this work, see the correlations between Monica and all the others person we have on the dataset.

First, we'll load the dataset and separate the data from Monica and from the others.

In [45]:
import pandas as pd

data = pd.read_csv('Netmovies_ratings_db.csv')

data.drop(columns=['timestamp'], inplace=True)

print(data)

            userName  rating                           title
0           Tom Cook     2.5                       Rocky III
1           Tom Cook     1.0                           Greed
2           Tom Cook     4.0                    American Pie
3           Tom Cook     2.0                        My Tutor
4           Tom Cook     2.0  Jay and Silent Bob Strike Back
...              ...     ...                             ...
44989  Charles Jones     4.0                   Boogie Nights
44990  Charles Jones     4.0                          Waiter
44991  Charles Jones     3.5                A Bridge Too Far
44992  Charles Jones     4.5                  The Last Laugh
44993  Charles Jones     4.0                           Miffo

[44994 rows x 3 columns]


In [46]:
def get_user_data(userName):
    return data[data['userName']==userName]
monica = get_user_data('Monica Villarreal')
data.drop(monica.index, inplace=True)
print(monica)

              userName  rating                               title
425  Monica Villarreal     1.0                 Million Dollar Baby
426  Monica Villarreal     3.0                          Predator 2
427  Monica Villarreal     5.0  Terminator 3: Rise of the Machines
428  Monica Villarreal     4.5           Monsieur  Hulot's Holiday
429  Monica Villarreal     5.0                    Dawn of the Dead
430  Monica Villarreal     5.0                    Cutthroat Island
431  Monica Villarreal     3.0                    Or (My Treasure)
432  Monica Villarreal     3.5                              Savior
433  Monica Villarreal     3.0                  Young and Innocent
434  Monica Villarreal     5.0                          La Chienne
435  Monica Villarreal     4.0    Ring of Fire II: Blood and Steel
436  Monica Villarreal     4.0                Heart of the Country


Since we have the Monica's data, we will have the opportunity to calculate the correlation of his rating with other users.

In [52]:
import numpy as np


users = data['userName'].unique()

correlations = {}
for user in users : 
    # Select data of the user  
    user_data = get_user_data(user)

    common_movies = list(set(monica['title']).intersection(set(user_data['title'])))
    if len(common_movies) > 1:  # We need at least two common movies to compute a correlation
        monica_common = monica[monica['title'].isin(common_movies)].drop_duplicates('title').set_index('title')
        user_common = user_data[user_data['title'].isin(common_movies)].drop_duplicates('title').set_index('title')
        monica_ratings = monica_common.loc[common_movies, 'rating']
        user_ratings = user_common.loc[common_movies, 'rating']
        corr = np.corrcoef(monica_ratings, user_ratings)[0, 1]
        correlations[user] = corr
    else:
        correlations[user] = None  # Not enough movies

print(correlations)


c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-p

{'Tom Cook': None, 'Katherine Stevens': None, 'Thomas Arnold': np.float64(0.9607689228305228), 'Karina Davis': np.float64(1.0), 'Michelle Roberson': None, 'Deanna Wise': None, 'Devin Davis': np.float64(nan), 'Edwin Welch': np.float64(-0.9999999999999999), 'Laurie Perez': None, 'Daniel Hopkins': None, 'Peter Brown': None, 'Melanie Hunter': np.float64(0.9999999999999998), 'Aimee Lee': None, 'Christopher Stevens': np.float64(0.9200129084349359), 'Timothy Ward': None, 'Maureen Hicks': np.float64(0.44022545316281186), 'Mark Duncan': None, 'Juan Ortiz': np.float64(0.17407765595569782), 'Frank Riddle': np.float64(-1.0), 'Timothy Lopez': np.float64(nan), 'John Miller': np.float64(1.0), 'Sarah Rice': np.float64(0.25318484177091666), 'Rebecca Vasquez': None, 'Shannon Curtis': None, 'Christopher Morales': np.float64(-0.9999999999999999), 'Marcus Cooper': np.float64(-0.9999999999999999), 'Gary Hughes': None, 'Mark Silva': None, 'John Marsh': np.float64(0.8703882797784891), 'Richard Hoover': np.flo

c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-p

We will now apply a thresold to identify the people that are the closest to Monica. The thresold is defined at 0.75.

In [53]:
users_selected = []

for user, corr in correlations.items():
    if corr != None :
        if corr > 0.95:
            users_selected.append(user)

print(users_selected)

['Thomas Arnold', 'Karina Davis', 'Melanie Hunter', 'John Miller', 'Richard Hoover', 'Deanna Bailey', 'Jeffrey Nelson', 'Brandon Brown', 'Kelly Hernandez', 'Brian Frank', 'Arthur Griffin', 'Chad Wheeler', 'Omar Smith', 'Lori Johnson', 'Kelly Williams', 'Joshua Graham', 'Katherine Martinez', 'Christopher Matthews', 'Monica Martinez', 'Joshua Henry', 'Denise Bryant', 'Robert Mitchell', 'Larry Taylor', 'Justin Rocha', 'Hector Watson MD', 'Joseph Cabrera', 'Maria Smith', 'Brian Davis', 'Robert Rios', 'Morgan Maldonado', 'Tanya Davis', 'Amanda Jones', 'Kathleen Parker', 'Erik Nelson', 'Alyssa Burns', 'Scott Steele', 'Joseph Hill', 'Jennifer Carter', 'Troy Harris', 'Anna Duncan', 'Matthew Miller', 'Robert Byrd', 'Anne Moss', 'Lisa Hudson', 'Olivia Bentley', 'Lee Hartman', 'Derrick Nash', 'Jonathan Richardson', 'Jacqueline Woodard', 'Mr. Michael Adams']
